## Detecting Moving Objects

There are many ways to track objects in a video, all of them fulfilling a slightly
different purpose. For example, you may want to track anything that moves, in
which case differences between frames are going to be of help; you may want to
track a hand moving in a video, in which case Meanshift based on the color of the
skin is the most appropriate solution; you may want to track a particular object of
which you know the aspect, in which case techniques such as template matching
will be of help.

<br><br><br><br>

## Basic Motion Detection

The first and most intuitive solution is to calculate the differences between frames, or
between a frame considered "background" and all the other frames.

In [1]:
import cv2
import numpy as np

camera = cv2.VideoCapture("resources/cars.mp4")
background = None

es = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9,4)) # for dilate 

while (True):
    ret, frame = camera.read()

    # set first frame as a background
    if background is None:
        background = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        background = cv2.GaussianBlur(background, (21, 21), 0)
        continue

    # convert grayscale and blur image
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (21, 21), 0) # in each video feed there will be noise and that noises may be detected as motion  to prevent it use cv2.GaussianBlur

    # calculate differences
    diff = cv2.absdiff(background, gray_frame)
    diff = cv2.threshold(diff, 50, 255, cv2.THRESH_BINARY)[1] # it returns two parameter first one is useless
    diff = cv2.dilate(diff, es, iterations = 2) # dilating can also act as a noise filter
    
    cnts, hierarchy = cv2.findContours(diff.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for c in cnts:
        if cv2.contourArea(c) < 1500: # only display contours for rectangles greater than an arbitrary threshold
             continue
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("contours", frame)
    cv2.imshow("dif", diff)

    if cv2.waitKey(17) & 0xff == ord("q"):
        break
    
cv2.destroyAllWindows()
camera.release()

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


• cv2.findContours: This function computes the contours of subjects in an image<br>
• cv2.boundinRect: This function calculates their bounding box

Conclusion :
this technique is quite fast to implement but it is not that suitable for applications <br>
because first you need to set deafult frame as a background and probably background is not going to stay constant in your applications .<br> Imagine your are detecting cars , setting deafult background is not going to effective because cars are moving everything is changing , light is changing ....<br>
That's where Background subtractors come into play

<br><br><br><br> 

##  Background subtractors – KNN, MOG2, and GMG

There are three background subtractors available in OpenCV 3: 
1. K-Nearest Neighbors (KNN) ++ 
2. Mixture of Gaussians (MOG2) ++  
3. Geometric Multigrid (GMG) -- 

BackgroundSubtractor classes are specifically built with video analysis in mind, which means that the OpenCV BackgroundSubtractor classes "learn" something about the environment with every frame<br><br>
Another fundamental (and frankly, quite amazing) feature of the BackgroundSubtractor classes is the ability to compute shadow . This is absolutely vital for an accurate reading of video frames; by detecting shadows, you can exclud shadow areas (by thresholding them) from the objects you detected, and concentra e on the real featuress

<br><br><br>

comparison : https://www.jetir.org/papers/JETIR1805563.pdf

In [10]:
# import libraries
import cv2
import numpy as np

# KNN
KNN_subtractor = cv2.createBackgroundSubtractorKNN(detectShadows = True) # detectShadows=True : exclude shadow areas from the objects you detected

# MOG2
MOG2_subtractor = cv2.createBackgroundSubtractorMOG2(detectShadows = True) # exclude shadow areas from the objects you detected

# choose your subtractor
bg_subtractor=MOG2_subtractor

camera = cv2.VideoCapture("resources/cars.mp4")

while True:
    ret, frame = camera.read()

    # Every frame is used both for calculating the foreground mask and for updating the background. 
    foreground_mask = bg_subtractor.apply(frame)

    # threshold if it is bigger than 240 pixel is equal to 255 if smaller pixel is equal to 0
    # create binary image , it contains only white and black pixels
    ret , treshold = cv2.threshold(foreground_mask.copy(), 120, 255,cv2.THRESH_BINARY)
    
    # reduce noise
    dilated = cv2.dilate(treshold,cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3)),iterations = 2)
    
     # find contours 
    contours, hier = cv2.findContours(dilated,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # check every contour if are exceed certain value draw bounding boxes
    for contour in contours:
        # if area exceed certain value then draw bounding boxes
        if cv2.contourArea(contour) > 350:
            (x,y,w,h) = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255, 255, 0), 2)

    cv2.imshow("Subtractor", foreground_mask)
    cv2.imshow("threshold", treshold)
    cv2.imshow("dilated", dilated)
    cv2.imshow("detection", frame)
    
    if cv2.waitKey(30) & 0xff == 27:
        break
        
camera.release()
cv2.destroyAllWindows()

In [10]:
import cv2
import numpy as np

In [11]:
cv2.createBackgroundSubtractorMOG2();
cv2.createBackgroundSubtractorKNN();

In [14]:
import cv2
import numpy as np

# KNN
KNN_subtractor = cv2.createBackgroundSubtractorKNN(detectShadows = True) # exclude shadow areas from the objects you detected

# MOG
MOG2_subtractor = cv2.createBackgroundSubtractorMOG2(detectShadows = True) # exclude shadow areas from the objects you detected

# choose your subtractor
bg_subtractor=MOG2_subtractor

camera = cv2.VideoCapture("resources/bird.mp4")

frame_width = int(camera.get(3)) 
frame_height = int(camera.get(4)) 
   
size = (frame_width, frame_height) 
   
# Below VideoWriter object will create 
# a frame of above defined The output  
# is stored in 'filename.avi' file. 
result = cv2.VideoWriter('birds.avi',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         10, size) 

while True:
    ret, frame = camera.read()

    # Every frame is used both for calculating the foreground mask and for updating the background. 
    foreground_mask = bg_subtractor.apply(frame)

    # threshold if it is bigger than 240 pixel is equal to 255 if smaller pixel is equal to 0
    ret , treshold = cv2.threshold(foreground_mask.copy(), 240, 255,cv2.THRESH_BINARY)
    
    dilated = cv2.dilate(treshold,cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3)),iterations = 2)
    
    contours, hier = cv2.findContours(dilated,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) > 100:
            (x,y,w,h) = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 0, 255), 3)

    cv2.imshow("Subtractor", foreground_mask)
    cv2.imshow("threshold", treshold)
    cv2.imshow("detection", frame)

    result.write(frame)
    
    if cv2.waitKey(30) & 0xff == 27:
        break
        
camera.release()
cv2.destroyAllWindows()

<br><br>

<br><br><br><br><br><br>

## Meanshift

Meanshift is an algorithm that tracks objects by finding
the maximum density of a discrete sample of a probability function (in our case, a
region of interest in an image) and recalculating it at the next frame, which gives the
algorithm an indication of the direction in which the object has moved.

2 very important built-in functions of 
OpenCV: calcHist and calcBackProject.

#### calcHist

The function, calcHist, calculates color histograms of an image .<br>
A color histogram 
is a representation of the color distribution of an image. On the x axis of th 
representation, we have color values, and on the y axis, we have the numb r
of pixels corresponding to the color values.

#### calcBackProject

The other function that covers a vital role in the Meanshift algorithm (but not only this) is calcBackProject, which is short for histogram back projection (calculation)<br>
A histogram back projection is so called because it takes a histogram and projects
it back onto an image, with the result being the probability that each pixel will
belong to the image that generated the histogram in the first place<br>
Therefore,
calcBackProject gives a probability estimation that a certain image is equal or
similar to a model image (from which the original histogram was generated).

cv2.calcBackProject() işlevi, bir görüntüdeki bir nesnenin olası konumunu tahmin etmek için kullanılan bir histogram temelli geri projeksiyon yöntemini uygular. Bu işlevin çıktısı, nesnenin olası konumunu belirlemek için kullanılan bir olasılık haritasıdır.

Bu olasılık haritası, nesnenin bulunma olasılığını belirtir ve genellikle piksel değerleri veya belirli bölgelerin yoğunluğu ile temsil edilir. Yani, çıktı bir koordinat değil, her pikselin nesnenin olası konumuna olan katkısını gösteren bir 2D görüntüdür.

Daha sonra, bu olasılık haritası, cv2.meanShift() gibi bir izleme algoritması tarafından kullanılabilir ve nesnenin mevcut konumu hesaplanabilir. Bu algoritma, belirli bir başlangıç konumundan hareket ederek olasılık haritasındaki maksimum yoğunluğa yönelir ve bu şekilde nesnenin konumunu belirler.

#### Conclusion

The calcHist function extracts a color histogram from an image, giving a statistical
representation of the colors in an image, and calcBackProject helps in calculating
the probability of each pixel of an image belonging to the original image.

!! Problem of Meanshift : : the size of the window does not change with the size of the object in the
frames being tracked<br>
CAMShift solves that problem.

meanshift algorithm example : <br>
https://github.com/siromermer/Object-Tracker-Meanshift-Algorithm/blob/main/meanshift.py

<BR><BR><BR>

## CAMShift

its implementation is similar to the meanshift , main difference is : after the call to CamShift,
the rectangle is drawn with a particular rotation that follows the rotation of the
object being tracked

camshift example : https://github.com/siromermer/Object-Tracker-Meanshift-Algorithm/blob/main/camshift.py

<br><br><br><br><br><br>